In [1]:
import pandas as pd
import numpy as np
import openpyxl

In [2]:
def open_task_file(name, sheet):
    # Открываем файл
    frame = None
    try:
        frame = pd.read_excel(name, sheet, skiprows=[0])
    except ValueError as err:
        if str(err) == 'Value must be either numerical or a string containing a wildcard':
            raise Exception(apply_filter_error_message)
        else:
            raise ValueError(err)
    return frame

def create_task_file():
    global graph_file_name, graph_sheet_name, week, who, ppr
    # Формирование заданий
    task_cols_names = ['Узел', 'Название работы', 'Периодичность', 'Плановое время', 'Фактическое время', 'Исполнитель', 'Дата выполнения', 'Комментарии']
    # Добавляем новые столбцы в исходную таблицу
    new_cols = ['Плановое время', 'Фактическое время', 'Исполнитель', 'Дата выполнения', 'Комментарии']
    for col in new_cols:
        ppr[col] = np.nan
    # Формируем таблицы с еженедельным рабочим заданием
    # Прихотливым индексированием выбираем только те строчки где есть галочка в соответствующей неделе и где соответствующий столбец 'Кто'
    week_who_df = ppr.loc[  
        (ppr.loc[:, week] == '✓') & (ppr.loc[:, 'Кто'] == who),
        ['Узел', 'ПУНКТ ТО / НЕДЕЛЯ', 'ПЕРИОДИЧНОСТЬ', 'Плановое время', 'Фактическое время', 'Исполнитель', 'Дата выполнения', 'Комментарии', 'Кто']
    ]
    # Переименовываем колонки (столбец 'Кто' в итоговой таблице не нужен, он нужен только по группировке)
    week_who_df.columns = task_cols_names + ['Кто']
    # Группируем по узлам
    with pd.ExcelWriter(f'рабочее_задание_неделя_{week}_{who}.xlsx') as writer:
        for node in week_who_df['Узел'].dropna().unique():
            week_who_node_df = week_who_df[week_who_df['Узел'] == node]
            info_df = pd.DataFrame([
                ["Наименование оборудования", node],
                ["N недели", week],
                ["Плановая группа", who],
                ["Подтверждение ответственного", np.nan],
                ["Подтверждение руководителя", np.nan]
            ])
            info_df.to_excel(writer, sheet_name=f'нед_{week}_{who}_{node}', header=False, index=False)
            week_who_node_df = week_who_node_df.reset_index()
            week_who_node_df = week_who_node_df[task_cols_names] #Убираем лишние столбцы('Кто')
            week_who_node_df.insert(0, 'N пп', week_who_node_df.index + 1)
            week_who_node_df.to_excel(writer, sheet_name=f'нед_{week}_{who}_{node}', startrow = len(info_df)+1, index=False)  # записываем без колонки кто
            #Форматируем лист по ширине
            worksheet = writer.sheets[f'нед_{week}_{who}_{node}']
            for col in worksheet.columns:
                max_length = 0
                column = col[0].column_letter
                max_length = max([len(str(cell.value)) for cell in col if cell.value != None])
                # перенос текста
                if (max_length > 100):
                    worksheet.column_dimensions[column].width = 50
                    for cell in col:
                        cell.alignment = openpyxl.styles.Alignment(wrap_text=True)
                # автоподбор ширины столбца
                else:
                    worksheet.column_dimensions[column].width = max_length+2
graph_file_name = 'График ТО ГА 2024.xlsx'
graph_sheet_name = 'ГОД'

apply_filter_error_message = 'Из-за применённых в файле фильтров, файл не может быть считан.  \
    Выставите все фильтры на (Выделить всё) и сохраните файл'

week = 1
who = 'м'


#Открываем файл
ppr = open_task_file(graph_file_name, graph_sheet_name)
# Переводим столбец 'Кто' в нижний регистр (игнорируем np.nan)
ppr.loc[~ppr['Кто'].isna(), 'Кто'] = ppr.loc[~ppr['Кто'].isna(), 'Кто'].map(lambda x : x.lower())
create_task_file()